# Introduction to MPI with mpi4py

MPI (abbreviation for message passing interface) is the standard library for communication within a cluster of processors. The library is available for many languages, such as C or Python. In this tutorial we use MPI within jupyter notebooks

## The installation happens in three steps:
* install MPI including mpi4py
* install PETSc including petsc4py (needed for advanced tutorials)
* install MPI-parallel ngsolve (shipped as binaries starting from NGSolve 24.04)

### Installing with conda

The quickest path is using conda. Conda-forge provides binary packages for MPI and PETSc for many platforms and versions (TODO: list versions). Install anaconda (or mini-conda), and run something like

    ~/miniconda3/bin/conda install mpi4py petsc4py
    ~/miniconda3/bin/python3 -m pip install --pre --upgrade ngsolve


### Installing conda-packages using pip

* *Linux/MacOS* <br>

      pip3 install -i https://pypi.anaconda.org/mpi4py/simple mpi4py==4.0.0.dev0 openmpi


* *Windows* <br>

      pip install impi_rt
      pip install -i https://pypi.anaconda.org/mpi4py/simple mpi4py==4.0.0.dev0
      

### Installing MPI and PETSc without conda

* *Linux:* <br>
  install either one with your package manager: openmpi, mpich, or IntelMPI <br>
  pip install mpi4py
  
* *MacOS:*

      download [openmpi4.1.6](https://www.open-mpi.org//software/ompi/v4.1/)
      ./configure
      make all
      sudo make all install
      pip install mpi4py 

  
* *Windows:* <br>
  Install IntelMPI


Install PETSc from a source-wheel:

    export PETSC_CONFIGURE_OPTIONS="--with-fc=0 --with-debugging=0 --download-hypre"
    pip cache remove petsc 
    pip install --upgrade --no-deps --force-reinstall mpi4py petsc petsc4py


## Using ipyparallel

The `ipyparallel` module let us communicate with the cluster. The `Cluster` class represents the cluster. Every processor starts its own Python instance. You have to install the ipyparallel package:

    !pip install ipyparallel

In [ ]:
from ipyparallel import Cluster
c = await Cluster(engines="mpi").start_and_connect(n=4, activate=True)
c.ids

In [ ]:
import mpi4py.MPI

We can define variables on the i$^{th}$ process:

In [ ]:
for i in c.ids:
    c[i]['a'] = i

Cells tagged with the %%px - magic are executed by the cluster processes:

In [ ]:
%%px
b = a*a

query the results from the processes:

In [ ]:
c[:]['b']

The MPI library
---

MPI provides functions to communicate within the cluster. The back-bone is a communicator object which knows about the participating processors. The standard communicator is the world-communicator where all processors of the cluster are included. 

The communicator knows the own number (called rank) within this group, and the size of the group:

In [ ]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD
print ('I am proc', comm.rank, 'within the team of', comm.size)

We can send messages between processes using `send` and `recv`. With send we give the destination process where to send, and with recv we give the source processor number from where we expect data. We can send one Python object, which may also be a tuple or list.

In this example, process $i$ is sending data to all processes with $j > i$. Then process $i$ is expecting data from processes with smaller ranks. This kind of communication is called point-to-point communication. Depending on the implementation of the mpi-library, and the object, the send and recv may or may not block.

In [ ]:
%%px
fruits = ['apple', 'banana', 'clementine', 'durian', \
          'elderberries', 'figs', 'grapes', 'honeydew melon']
for dst in range(comm.rank+1, comm.size):
    comm.send(fruits[comm.rank%8], dest=dst)
for src in range(comm.rank):
    print ("got a", comm.recv(source=src))

If we want to send data to all processes in the cluster we use collective communication. A broadcast operation sends the same data from the root to everyone, a scatter operation splits a list to all processes. The default root is process 0.

In [ ]:
%%px
if comm.rank == 0:
    comm.bcast("Hello from the boss!!")
    comm.scatter( ["personal note to "+str(i) for i in range(comm.size)])
else:
    print (comm.bcast(None))
    print (comm.scatter(None))

The technology behind sending of Python objects is pickling. Every class which knows how to convert itself to and from a byte-stream (called serialization) can be exchanged via mpi communication.

In [ ]:
c.shutdown(hub=True)